1. Introdução:

Este notebook tem o intuito de relacionar as bases 1 e 2 conforme descritos abaixo:
    
    1. "Emission_Country_Year" - Base principal do projeto, pois contén os dados quantitativos relacionados a emissão de gases poluentes por País e Ano, esta base registra dados de 1850 a 2019 de 236 países.

    2. "Kyoto_Agreement_Countries" - Base auxiliar do projeto, pois contém informações de países que assinaram o tratado de kyoto, esta base registra dados de 109 países. 

As relações das bases se darão pelas colunas Base 1, Coluna "country" e Base 2, coluna "Participant" que são comuns em ambas as bases, Foram necessárias algumas trtativas na Base 2 a fim de deixa-la o mais proximo possível da Base 1.

Contextualização: O tratado de Kyoto é um tratado internacional com compromissos mais rígidos para a redução da emissão dos gases que produzem o efeito estufa, que são a causa do atual aquecimento global, o tratado foi assinado em 1997 porém so entrou em vigor em 2005, o tratado expirou em 2012.

In [1]:
#Importanção de pacortes necessários
import pandas as pd
import requests
from bs4 import BeautifulSoup

1. Base de Dados 2 - "Kyoto_Agreement_Countries"

In [2]:
#Requisição de dados no site para coleta de dados e cração da base 2. Kyoto_Agreement_Countries
reqkyoto = requests.get('https://treaties.un.org/Pages/ViewDetails.aspx?src=TREATY&mtdsg_no=XXVII-7-a&chapter=27&clang=_en')
if reqkyoto.status_code == 200:
    print('Requisição bem sucedida!')
    contentkyoto = reqkyoto.content

Requisição bem sucedida!


In [3]:
#criação de lista com dados requisitados anteriormente
soupkyoto = BeautifulSoup(contentkyoto, 'html.parser')
#buscando a tag table na lista, já que os dados que vamos consumir está em uma tabela neste site.
tablekyoto = soupkyoto.find(name='table')

In [4]:
#transformando lista em string
table_str_kyoto = str(tablekyoto)
#lendo o 10° item da lista e o convertendo em dataframe, obs: foi testado varias posiçoes da lista e identificado que a tabela 10 é exatamente a tabela necessitada.
Kyoto_Agreement_Countries = pd.read_html(table_str_kyoto)[10:11][0]
#exibição do dataframe
Kyoto_Agreement_Countries

,0,1,2
0,Participant,Signature,"Ratification, Acceptance(A), Accession(a), App..."
1,Afghanistan,NaN,25 Mar 2013 a
2,Albania,NaN,1 Apr 2005 a
3,Algeria,NaN,16 Feb 2005 a
4,Angola,NaN,8 May 2007 a
...,...,...,...
190,Venezuela (Bolivarian Republic of),NaN,18 Feb 2005 a
191,Viet Nam,3 Dec 1998,25 Sep 2002
192,Yemen,NaN,15 Sep 2004 a
193,Zambia,5 Aug 1998,7 Jul 2006


Tratativa da base de dados "Base 2. Kyoto_Agreement_Countries":

A Coluna Signature se refere a data em que o país assinou o tratado de Kyoto, na coluna Date trata-se de diversas ações possívesi após a assinatura como:
   1. Approval(AA)
   2. Acceptance(A)
   3. Accession(a)
   4. Ratification
  
A informação mais importante nesse projeto é a adesão e não apenas aprovação, aceitação ou ratificação, pois deseja-se descobrir quais países aderiram ao protocolo e implantaram mudanças para o atingimento de metas, por este motivo todos os paises que não tenham a marcação "a" na coluna "Date" serão descartados.

In [5]:
#Drop na linha 0 do dataframe pois trata-se dos nomes da coluna, em caso de dúvidas olhar na visualização acima na linha de número 0
Kyoto_Agreement_Countries.drop(0, axis=0, inplace=True)
#inserção dos nomes das colunas no dataframe
Kyoto_Agreement_Countries.columns=['Participant','Signature','Date']
#Criação da coluna 'Accession(a)' que vale True se a coluna 'Date' finaliza com ' a'
Kyoto_Agreement_Countries['Accession(a)'] = Kyoto_Agreement_Countries.Date.str.endswith(" a")
#exibição do dataframe
Kyoto_Agreement_Countries

,Participant,Signature,Date,Accession(a)
1,Afghanistan,NaN,25 Mar 2013 a,True
2,Albania,NaN,1 Apr 2005 a,True
3,Algeria,NaN,16 Feb 2005 a,True
4,Angola,NaN,8 May 2007 a,True
5,Antigua and Barbuda,16 Mar 1998,3 Nov 1998,False
...,...,...,...,...
190,Venezuela (Bolivarian Republic of),NaN,18 Feb 2005 a,True
191,Viet Nam,3 Dec 1998,25 Sep 2002,False
192,Yemen,NaN,15 Sep 2004 a,True
193,Zambia,5 Aug 1998,7 Jul 2006,False


In [6]:
# indexa sempre que a coluna 'Accession(a)' valer false, ou seja, quando não aderido.
indexNames = Kyoto_Agreement_Countries[(Kyoto_Agreement_Countries['Accession(a)'] == False)].index 
# Deletar linhas conforme indexação
Kyoto_Agreement_Countries.drop(indexNames, inplace=True)
# Deletar a coluna 'Signature'
Kyoto_Agreement_Countries.drop(['Signature'], axis=1, inplace=True)
#exibição do dataframe
Kyoto_Agreement_Countries

,Participant,Date,Accession(a)
1,Afghanistan,25 Mar 2013 a,True
2,Albania,1 Apr 2005 a,True
3,Algeria,16 Feb 2005 a,True
4,Angola,8 May 2007 a,True
7,Armenia,25 Apr 2003 a,True
...,...,...,...
186,United States of America,NaN,NaN
189,Vanuatu,17 Jul 2001 a,True
190,Venezuela (Bolivarian Republic of),18 Feb 2005 a,True
192,Yemen,15 Sep 2004 a,True


Limpeza da base de dados:

In [7]:
#Substitui a string ' a' por '' na coluna 'Date' quando a mesma finalizar com este valor 
Kyoto_Agreement_Countries.Date = Kyoto_Agreement_Countries.Date.str.replace(' a$', '')
#Substitui a string '[' por '' na coluna 'Date' quando a mesma contiver este valor 
Kyoto_Agreement_Countries.Date = Kyoto_Agreement_Countries.Date.str.replace('[', '')
#Substitui a string '[' por '' na coluna 'Date' quando a mesma contiver este valor 
Kyoto_Agreement_Countries.Date = Kyoto_Agreement_Countries.Date.str.replace(' ]', '')
#Substitui a string ' ' por '' na coluna 'Date' quando a mesma contiver este valor 
Kyoto_Agreement_Countries.Date = Kyoto_Agreement_Countries.Date.str.replace(' $', '')
#Drop todos os valores nulos do dataframe
Kyoto_Agreement_Countries.dropna(inplace=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [8]:
#Converte a coluna 'Date' em data
Kyoto_Agreement_Countries['Date'] = pd.to_datetime(Kyoto_Agreement_Countries['Date'])
#Corrigido os nomes na coluna 'Participant' pois havia nomes com '()'
Kyoto_Agreement_Countries['Participant'] = Kyoto_Agreement_Countries['Participant'].str.split('(').apply(lambda x : x[0])
#Substitui a string ' ' por '' na coluna 'Participant' quando a mesma finalizar com este valor 
Kyoto_Agreement_Countries.Participant = Kyoto_Agreement_Countries.Participant.str.replace(' $', '')
#Reset index
Kyoto_Agreement_Countries.reset_index(inplace = True)
#Drop coluna index
Kyoto_Agreement_Countries.drop(['index'],axis=1, inplace = True)
#exibição do dataframe
Kyoto_Agreement_Countries

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


,Participant,Date,Accession(a)
0,Afghanistan,2013-03-25,True
1,Albania,2005-04-01,True
2,Algeria,2005-02-16,True
3,Angola,2007-05-08,True
4,Armenia,2003-04-25,True
...,...,...,...
105,United Republic of Tanzania,2002-08-26,True
106,Vanuatu,2001-07-17,True
107,Venezuela,2005-02-18,True
108,Yemen,2004-09-15,True


2. Base de Dados 1 - "Emission_Country_Year"

In [9]:
#url dos dados que deseja-se explorar no github
url = 'https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv'
#Leitura da url com o pandas
Emission_Country_Year = pd.read_csv(url, error_bad_lines=False)

Tratamento da base

In [10]:
#dropping some columns that is not important for us, cleanig de dataset
Emission_Country_Year.drop(['co2_growth_prct','co2_growth_abs', 'consumption_co2', 'trade_co2', 'trade_co2_share','consumption_co2_per_capita','consumption_co2_per_gdp','co2_per_unit_energy','primary_energy_consumption','energy_per_capita','energy_per_gdp'],axis=1, inplace=True)
#Drop todos os valores nulos da coluna 'country' 
Emission_Country_Year.dropna(subset=['country'], inplace=True)
#Substitui a string ' ' por '' na coluna 'country' quando a mesma finalizar com este valor 
Emission_Country_Year.country = Emission_Country_Year.country.str.replace(' $', '')
Emission_Country_Year

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


,iso_code,country,year,co2,co2_per_capita,share_global_co2,cumulative_co2,share_global_cumulative_co2,co2_per_gdp,cement_co2,...,share_global_cumulative_flaring_co2,share_global_cumulative_cement_co2,total_ghg,ghg_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp
0,AFG,Afghanistan,1949,0.015,0.002,0.000,0.015,0.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7663783.0,NaN
1,AFG,Afghanistan,1950,0.084,0.011,0.001,0.099,0.000,0.004,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7752000.0,1.949480e+10
2,AFG,Afghanistan,1951,0.092,0.012,0.001,0.191,0.000,0.005,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7840000.0,2.006385e+10
3,AFG,Afghanistan,1952,0.092,0.012,0.001,0.282,0.000,0.004,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7936000.0,2.074235e+10
4,AFG,Afghanistan,1953,0.106,0.013,0.002,0.388,0.000,0.005,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8040000.0,2.201546e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23703,ZWE,Zimbabwe,2015,12.170,0.881,0.035,726.480,0.048,0.486,0.504,...,NaN,0.049,67.49,4.885,11.87,0.859,6.68,0.484,13815000.0,2.503057e+10
23704,ZWE,Zimbabwe,2016,10.815,0.771,0.031,737.295,0.048,0.430,0.581,...,NaN,0.049,65.98,4.703,11.92,0.850,6.55,0.467,14030000.0,2.515176e+10
23705,ZWE,Zimbabwe,2017,10.247,0.720,0.029,747.542,0.047,NaN,0.581,...,NaN,0.048,NaN,NaN,NaN,NaN,NaN,NaN,14237000.0,NaN
23706,ZWE,Zimbabwe,2018,11.341,0.785,0.031,758.882,0.047,NaN,0.581,...,NaN,0.048,NaN,NaN,NaN,NaN,NaN,NaN,14439000.0,NaN


In [11]:
#Criação de dataframe com 2 colunas de Emission_Country_Year
Country_IsoCode = Emission_Country_Year[['iso_code', 'country', 'year']]
#Drop todos os valores nulos da coluna 'country' 
Country_IsoCode.dropna(subset=['country'], inplace=True)
#exibição do dataframe
Country_IsoCode

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,iso_code,country,year
0,AFG,Afghanistan,1949
1,AFG,Afghanistan,1950
2,AFG,Afghanistan,1951
3,AFG,Afghanistan,1952
4,AFG,Afghanistan,1953
...,...,...,...
23703,ZWE,Zimbabwe,2015
23704,ZWE,Zimbabwe,2016
23705,ZWE,Zimbabwe,2017
23706,ZWE,Zimbabwe,2018


In [12]:
Country_IsoCode['iso_code'].fillna('Preencher',inplace=True)
#Agrupamento por 'iso_code' selecionando os dados da tabela 'country' index resetado para 'iso_code' e o index virou a coluna
Country_IsoCode = Country_IsoCode.groupby('country')['iso_code'].apply(lambda Country_IsoCode: Country_IsoCode.reset_index(drop=True)).unstack(0)
#Drop todos os valores nulos
Country_IsoCode.dropna(inplace = True)
#exibição do dataframe
Country_IsoCode

/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:4439: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


country,Afghanistan,Africa,Albania,Algeria,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,Wallis and Futuna Islands,World,Yemen,Zambia,Zimbabwe
0,AFG,Preencher,ALB,DZA,AND,AGO,AIA,ATG,ARG,ARM,...,URY,UZB,VUT,VEN,VNM,WLF,OWID_WRL,YEM,ZMB,ZWE


In [13]:
#dataframe transposto
Country_IsoCode = Country_IsoCode.transpose()
#reset de index
Country_IsoCode.reset_index(inplace = True)
#Substitui a string ' ' por '' na coluna 'country' quando a mesma finalizar com este valor 
Country_IsoCode.country = Country_IsoCode.country.str.replace(' $', '')
#Renomear nome das colunas
Country_IsoCode.columns=['Country','Iso_code']
#exibição do dataframe
Country_IsoCode

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


,Country,Iso_code
0,Afghanistan,AFG
1,Africa,Preencher
2,Albania,ALB
3,Algeria,DZA
4,Andorra,AND
...,...,...
231,Wallis and Futuna Islands,WLF
232,World,OWID_WRL
233,Yemen,YEM
234,Zambia,ZMB


In [14]:
#verificando quantos valores da coluna 'Country são nulos'
Country_IsoCode['Country'].nunique()

236

3. Alteração na base "Base 2. Kyoto_Agreement_Countries", Conforme Base 1. "Emission_Country_Year".

In [15]:
#Substituição dos nomes dos países a fim de deixar a base conforme a base principal do projeto 
Kyoto_Agreement_Countries.Participant = Kyoto_Agreement_Countries.Participant.str.replace('Brunei Darussalam', 'Brunei')
Kyoto_Agreement_Countries.Participant = Kyoto_Agreement_Countries.Participant.str.replace('Cabo Verde', 'Cape Verde')
Kyoto_Agreement_Countries.Participant = Kyoto_Agreement_Countries.Participant.str.replace("Côte d'Ivoire", "Cote d'Ivoire")
Kyoto_Agreement_Countries.Participant = Kyoto_Agreement_Countries.Participant.str.replace('Democratic Republic of the Congo', 'Democratic Republic of Congo')
Kyoto_Agreement_Countries.Participant = Kyoto_Agreement_Countries.Participant.str.replace("Lao People's Democratic Republic", 'Laos')
Kyoto_Agreement_Countries.Participant = Kyoto_Agreement_Countries.Participant.str.replace('Republic of Moldova', 'Moldova')
Kyoto_Agreement_Countries.Participant = Kyoto_Agreement_Countries.Participant.str.replace('St. Kitts and Nevis', 'Saint Kitts and Nevis')
Kyoto_Agreement_Countries.Participant = Kyoto_Agreement_Countries.Participant.str.replace('Syrian Arab Republic', 'Syria')
Kyoto_Agreement_Countries.Participant = Kyoto_Agreement_Countries.Participant.str.replace("Democratic People's Republic of Korea", 'North Korea')
Kyoto_Agreement_Countries.Participant = Kyoto_Agreement_Countries.Participant.str.replace('Timor-Leste', 'Timor')
Kyoto_Agreement_Countries.Participant = Kyoto_Agreement_Countries.Participant.str.replace('United Republic of Tanzania', 'Tanzania')
#exibição do dataframe
Kyoto_Agreement_Countries

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  


,Participant,Date,Accession(a)
0,Afghanistan,2013-03-25,True
1,Albania,2005-04-01,True
2,Algeria,2005-02-16,True
3,Angola,2007-05-08,True
4,Armenia,2003-04-25,True
...,...,...,...
105,Tanzania,2002-08-26,True
106,Vanuatu,2001-07-17,True
107,Venezuela,2005-02-18,True
108,Yemen,2004-09-15,True


Drop de valores que não existiam na base de "1. Emission_Country_Year"

In [16]:
# Get indexes where name column doesn't have value john
indexNames = Kyoto_Agreement_Countries[(Kyoto_Agreement_Countries['Participant'] == 'San Marino')].index 
# Delete these row indexes from dataFrame
Kyoto_Agreement_Countries.drop(indexNames, inplace=True)
#exibição do dataframe
Kyoto_Agreement_Countries

,Participant,Date,Accession(a)
0,Afghanistan,2013-03-25,True
1,Albania,2005-04-01,True
2,Algeria,2005-02-16,True
3,Angola,2007-05-08,True
4,Armenia,2003-04-25,True
...,...,...,...
105,Tanzania,2002-08-26,True
106,Vanuatu,2001-07-17,True
107,Venezuela,2005-02-18,True
108,Yemen,2004-09-15,True


4. Análise de dados Verificação se as bases 1 e 2 estão correspondentes:

Foram Analisadas as colunas Kyoto_Agreement_Countries['Participant'] e Country_IsoCode['Country'] para verificação de correspondencias.
Foram criados 3 dataframes conforme abaixo:
    
    1. kyoto_naoIsocode: Países existentes na base de dados "2. Kyoto_Agreement_Countries" porém não na base "1. Emission_Country_Year", foram encontrados 12 valores na primeira iteração, porém houve verificação na escrita destes países e houve alterações de nomes, conforme item 2.3, após estas alterações encontrou-se apenas 1 valor, como a base 2 é apenas uma base auxiliar este país foi excluído da base 2, conforme iteração 3.

    1° Iteração Resultado:

In [17]:
#Dataframe criado com as linhas em que a coluna 'Participant' nao estava em Country_IsoCode['country'] nesse caso sera necessario substituir
#Kyoto_Agreement_Countries['Participant'] pela sua base e coluna a qual contem o nome dos países
kyoto_naoIsocode = Kyoto_Agreement_Countries[Kyoto_Agreement_Countries['Participant'].isin(Country_IsoCode['Country']) == False]
#contando o numero de linhas nao casadas
kyoto_naoIsocode.count()

Participant     0
Date            0
Accession(a)    0
dtype: int64

In [18]:
kyoto_naoIsocode

,Participant,Date,Accession(a)


    2. kyoto_Isocode: Países existentes em ambas as base, foram encontrados 98 valores na primeira iteração, porém após as alterações na base conforme item 2.3 esse valor subiu para 109 países, ou seja, 109 países da base geral eceitaram implementar ações para diminuição de emissão de gases poluentes.

In [19]:
#Dataframe criado com as linhas em que a coluna 'Participant' existe em, nesse caso temos o match e nao sera necessario se preocupar com estas linhas
kyoto_Isocode = Kyoto_Agreement_Countries[Kyoto_Agreement_Countries['Participant'].isin(Country_IsoCode['Country'])]
#contando o numero de linhas casadas entre os dataframes
kyoto_Isocode.count()

Participant     109
Date            109
Accession(a)    109
dtype: int64

In [20]:
kyoto_Isocode

,Participant,Date,Accession(a)
0,Afghanistan,2013-03-25,True
1,Albania,2005-04-01,True
2,Algeria,2005-02-16,True
3,Angola,2007-05-08,True
4,Armenia,2003-04-25,True
...,...,...,...
105,Tanzania,2002-08-26,True
106,Vanuatu,2001-07-17,True
107,Venezuela,2005-02-18,True
108,Yemen,2004-09-15,True


    3. naokyoto_Isocode: Países existentes na base de dados "1. Emission_Country_Year" porém não na base "2. Kyoto_Agreement_Countries", foram encontrados 138 valores na primeira iteração, porém após as alterações na base conforme item 2.3 esse valor diminuiu para 127 países, ou seja, 127 países da base "1. Emission_Country_Year" não implementaram ações para a diminuição na emissão de gases poluentes. 

In [21]:
#Dataframe criado com as linhas em que a coluna Country_IsoCode['country'] nao estava em Kyoto_Agreement_Countries['Participant'] nesse caso sera necessario substituir
#Kyoto_Agreement_Countries['Participant'] pela sua base e coluna a qual contem o nome dos países
naokyoto_Isocode = Country_IsoCode[Country_IsoCode['Country'].isin(Kyoto_Agreement_Countries['Participant']) == False]
#contando o numero de linhas nao casadas
naokyoto_Isocode.count()

Country     127
Iso_code    127
dtype: int64

In [22]:
naokyoto_Isocode

,Country,Iso_code
1,Africa,Preencher
4,Andorra,AND
6,Anguilla,AIA
7,Antigua and Barbuda,ATG
8,Argentina,ARG
...,...,...
227,Uzbekistan,UZB
230,Vietnam,VNM
231,Wallis and Futuna Islands,WLF
232,World,OWID_WRL


5. Conclusão

Será incluído uma coluna no Dataframe 1 com os Valores True e False onde true conrresponde aos países encontrados na base 2, que são os países que "Assinaram" o Tratado de Kyoto.

5.1 Inclusão de coluna na base de dados 1. Emission_Country_Year referente ao tratado de Kyoto.

In [23]:
#Criação da coluna 'kyoto' que vale True se 
Emission_Country_Year['Kyoto'] = Emission_Country_Year['country'].isin(Kyoto_Agreement_Countries['Participant'])
Emission_Country_Year

,iso_code,country,year,co2,co2_per_capita,share_global_co2,cumulative_co2,share_global_cumulative_co2,co2_per_gdp,cement_co2,...,share_global_cumulative_cement_co2,total_ghg,ghg_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,Kyoto
0,AFG,Afghanistan,1949,0.015,0.002,0.000,0.015,0.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7663783.0,NaN,True
1,AFG,Afghanistan,1950,0.084,0.011,0.001,0.099,0.000,0.004,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7752000.0,1.949480e+10,True
2,AFG,Afghanistan,1951,0.092,0.012,0.001,0.191,0.000,0.005,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7840000.0,2.006385e+10,True
3,AFG,Afghanistan,1952,0.092,0.012,0.001,0.282,0.000,0.004,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7936000.0,2.074235e+10,True
4,AFG,Afghanistan,1953,0.106,0.013,0.002,0.388,0.000,0.005,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8040000.0,2.201546e+10,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23703,ZWE,Zimbabwe,2015,12.170,0.881,0.035,726.480,0.048,0.486,0.504,...,0.049,67.49,4.885,11.87,0.859,6.68,0.484,13815000.0,2.503057e+10,True
23704,ZWE,Zimbabwe,2016,10.815,0.771,0.031,737.295,0.048,0.430,0.581,...,0.049,65.98,4.703,11.92,0.850,6.55,0.467,14030000.0,2.515176e+10,True
23705,ZWE,Zimbabwe,2017,10.247,0.720,0.029,747.542,0.047,NaN,0.581,...,0.048,NaN,NaN,NaN,NaN,NaN,NaN,14237000.0,NaN,True
23706,ZWE,Zimbabwe,2018,11.341,0.785,0.031,758.882,0.047,NaN,0.581,...,0.048,NaN,NaN,NaN,NaN,NaN,NaN,14439000.0,NaN,True


5.2 Salvamento de arquivos no Storage

In [24]:
#importação de pacote
from google.cloud import storage

In [25]:
# criação de um dataframe com a chave criada no IAM
key = pd.read_csv('/home/greyce_cos_sil/projetoteste-299919-abc73be2d5b5.json') 
#key

In [26]:
# setando as credeciais com o arquivo json que contem a chave
client = storage.Client.from_service_account_json(json_credentials_path='/home/greyce_cos_sil/projetoteste-299919-abc73be2d5b5.json')

In [27]:
# setando o bucket onde deseja-se salvar os arquivos
bucket = client.get_bucket('jupyter_note')

5.2.1 Dataframe 1. Emission_Country_Year

In [28]:
#salvando os dados do dataframe Kyoto_Agreement_Countries em um arquivo csv
Emission_Country_Year.to_csv("Emission_Country_Year.csv")

In [29]:
#Nome do arquivo que será salvo no bucket
object_Emission_Country_Year_bucket = bucket.blob('Emission_Country_Year.csv')

In [30]:
# Name of the object in local file system
object_Emission_Country_Year_bucket.upload_from_filename('/home/jupyter/base/Emission_Country_Year.csv')

5.2.2 Dataframe 2. Kyoto_Agreement_Countries

In [31]:
#salvando os dados do dataframe Kyoto_Agreement_Countries em um arquivo csv
Kyoto_Agreement_Countries.to_csv("Kyoto_Agreement_Countries.csv")

In [32]:
#Nome do arquivo que será salvo no bucket
object_Kyoto_Agreement_Countries_bucket = bucket.blob('Kyoto_Agreement_Countries.csv')

In [33]:
# Name of the object in local file system
object_Kyoto_Agreement_Countries_bucket.upload_from_filename('/home/jupyter/base/Kyoto_Agreement_Countries.csv')

Autor do notebook >>>>>>>>>>>>>>>>>>> Greyce da Costa Silva